1. yes24 best셀러 2008-2023 11월까지 수집(목록)
2. 수집한 자료에서 세부 데이터 수집 리뷰포함
3. 수집한 자료를 mysql에 저장하기, DB설계 필요(원격)
4. DB에서 자료 읽어오기
5. 추천시스템을 위한 장르, 키워드 추출 (정규화 (중복최소화))
6. 코사인 유사도로 추천 시스템 제작
7. 장르 및 키워드별 대표 책 추출 후 
   책 추천시스템 HTML 화면에 보여주고 
   유저가 책을 클릭하면 해당 장르의 책이 
   하단에 출력되도록 website제작

In [1]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬
from tqdm import tqdm
import sqlite3
from sqlalchemy import create_engine
import yes24_dbio as ydb

https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber=001&pageNumber=1&pageSize=&type=month

In [19]:
url = 'https://www.yes24.com/Product/Category/MonthWeekBestSeller'
payload = dict(categoryNumber='001', pageNumber='1', pageSize='120', type='month', saleYear=2023, saleMonth=12)
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.text, 'lxml')

https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber=001&pageNumber=1&pageSize=120&type=month&saleYear=2023&saleMonth=12
200


In [9]:
today = datetime.today()
yyyy = today.year
mm = today.month
print(yyyy, mm)

2023 12


In [16]:
year = list(range(2008, 2024))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [136]:
book_list = []
for bestYear in tqdm(range(2008,2024)):
    for bestMonth in range(1,13):
        for pageNum in range(1,10):
            url = f"https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber=001&pageNumber={pageNum}&pageSize=120&type=month&saleYear={bestYear}&saleMonth={bestMonth}"
            r = requests.get(url)
            soup = bs(r.text,"lxml")
            print(f"현재 {bestYear}년도 {bestMonth}월 {pageNum}페이지 크롤링중",end='\r')
            time.sleep(3)
            for book in soup.select("#yesBestList > li"):
                year = bestYear
                month = bestMonth
                book_img = book.select_one(".img_item>span>a>em>img")["data-original"] if book.select_one(".img_item>span>a>em>img") else ""
                title_h = book.select_one(".gd_res").get_text()
                title_f = book.select_one(".gd_nameF").get_text() if book.select_one(".gd_nameF") else ""
                title_m = book.select_one(".gd_name").get_text() if book.select_one(".gd_name") else "제목없음"
                title_e = book.select_one(".gd_nameE").get_text() if book.select_one(".gd_nameE") else ""
                detail_link = "https://www.yes24.com" + book.select_one(".gd_name")["href"] if book.select_one(".gd_name") else "제목없음"
                author = [i.get_text() for i in book.select(".authPub.info_auth > a")] if book.select(".authPub.info_auth > a") else []
                publisher = book.select_one(".authPub.info_pub > a").get_text() if book.select_one(".authPub.info_pub > a") else ""
                pub_date = book.select_one(".authPub.info_date").get_text() if book.select_one(".authPub.info_date") else ""
                price = book.select_one(".yes_b").get_text() if book.select_one(".yes_b") else 0
                n_reviews = book.select_one(".txC_blue").get_text() if book.select_one(".txC_blue") else 0
                review_link = book.select_one(".rating_rvCount > a")['href'] if book.select_one(".rating_rvCount > a") else ""
                rating = book.select_one("span.rating_grade > .yes_b").get_text() if book.select_one("span.rating_grade > .yes_b") else 0
                tags = [i.get_text() for i in book.select(".info_row.info_tag > .tag > a")] if book.select(".info_row.info_tag > .tag > a") else []

                book_list.append([year,month,book_img, title_h, title_f, title_m, title_e, detail_link, author, publisher, pub_date, price, n_reviews,
                     review_link, rating, tags])

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

  6%|█████                                                                           | 1/16 [11:42<2:55:34, 702.33s/it]

 12%|██████████                                                                      | 2/16 [22:55<2:39:53, 685.23s/it]

 19%|███████████████                                                                 | 3/16 [34:40<2:30:26, 694.36s/it]

 25%|████████████████████                                                            | 4/16 [46:19<2:19:15, 696.26s/it]

...

 75%|███████████████████████████████████████████████████████████▎                   | 12/16 [2:20:21<47:19, 709.83s/it]

 81%|████████████████████████████████████████████████████████████████▏              | 13/16 [2:32:47<36:02, 720.87s/it]

 88%|█████████████████████████████████████████████████████████████████████▏         | 14/16 [2:44:41<23:57, 718.61s/it]

 94%|██████████████████████████████████████████████████████████████████████████     | 15/16 [2:56:26<11:54, 714.52s/it]

100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [3:08:03<00:00, 705.23s/it]


In [201]:
df = pd.DataFrame(book_list)

In [202]:
df = df.drop(4, axis=1)

In [203]:
df.to_csv('./data/yes24_08-23월간베스트.csv', index=False)

In [3]:
data = pd.read_csv('./data/yes24_08-23월간베스트.csv')

C:\Users\hi\AppData\Local\Temp\ipykernel_1824\1187745429.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/yes24_08-23월간베스트.csv')


In [4]:
colnames = ['yy', 'mon', 'image', 'category', 'title', 'title_sub', 'link', 'author', \
         'publisher', 'pub_date', 'price', 'n_reviews', 'detail_link', 'rating', 'tags']

In [5]:
data.columns = colnames

In [6]:
data = data.drop('title_sub', axis=1)

In [7]:
data[data.duplicated('title')]

,yy,mon,image,category,title,link,author,publisher,pub_date,price,n_reviews,detail_link,rating,tags
219,2008,1,https://image.yes24.com/goods/2801867/L,[도서],내 아이의 천재성을 살려 주는 엄마표 홈스쿨링,https://www.yes24.com/Product/Goods/2801867,['진경혜'],중앙북스(books),2008년 01월,"8,550",23,https://www.yes24.com/Product/Goods/2801867?Re...,9.5,[]
473,2008,1,https://image.yes24.com/goods/2752647/L,[도서],네이티브는 쉬운 영어로 말한다,https://www.yes24.com/Product/Goods/2752647,['박수진'],길벗이지톡,2007년 11월,"8,820",10,https://www.yes24.com/Product/Goods/2752647?Re...,8.7,[]
695,2008,1,https://image.yes24.com/goods/2784581/L,[도서],네이티브는 쉬운 영어로 말한다,https://www.yes24.com/Product/Goods/2784581,['박수진'],길벗이지톡,2007년 12월,"8,820",4,https://www.yes24.com/Product/Goods/2784581?Re...,8.8,[]
742,2008,1,https://image.yes24.com/goods/2789758/L,[도서],화성에서 온 남자 금성에서 온 여자,https://www.yes24.com/Product/Goods/2789758,"['존 그레이', '김경숙']",동녘라이프,2007년 12월,"12,420",7,https://www.yes24.com/Product/Goods/2789758?Re...,6.3,[]
755,2008,1,https://image.yes24.com/goods/2714262/L,[도서],나무,https://www.yes24.com/Product/Goods/2714262,['이순원'],뿔(웅진문학에디션),2007년 10월,"8,100",137,https://www.yes24.com/Product/Goods/2714262?Re...,9.1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188959,2023,12,https://image.yes24.com/goods/117650984/L,[도서],초수익 성장주 투자,https://www.yes24.com/Product/Goods/117650984,"['마크 미너비니', '김태훈', '김대현']",이레미디어,2023년 03월,"22,500",49,https://www.yes24.com/Product/Goods/117650984?...,9.6,['#크레마클럽에있어요']
188960,2023,12,https://image.yes24.com/goods/117585533/L,[도서],상처 주는 말 하는 친구에게 똑똑하게 말하는 법,https://www.yes24.com/Product/Goods/117585533,"['김윤나', '유영근']",북라이프,2023년 02월,"11,700",52,https://www.yes24.com/Product/Goods/117585533?...,9.9,"['#학교생활이야기', '#슬기로운언어생활', '#친구관계', '#인간관계에대하여']"
188961,2023,12,https://image.yes24.com/goods/115033554/L,[도서],사람을 얻는 지혜,https://www.yes24.com/Product/Goods/115033554,"['그라시안 이 모랄레스 발타사르', '김유경']",현대지성,2022년 10월,"10,800",103,https://www.yes24.com/Product/Goods/115033554?...,9.6,['#크레마클럽에있어요']
188962,2023,12,https://image.yes24.com/goods/114848694/L,[도서],인생의 역사,https://www.yes24.com/Product/Goods/114848694,['신형철'],난다,2022년 10월,"16,200",111,https://www.yes24.com/Product/Goods/114848694?...,9.6,"['#2023서울국제도서전', '#YES24XEBS_대국민독서챌린지', '#2022..."


In [8]:
data["link"]

0           https://www.yes24.com/Product/Goods/2599166
1           https://www.yes24.com/Product/Goods/2776013
2           https://www.yes24.com/Product/Goods/2721480
3           https://www.yes24.com/Product/Goods/2783642
4           https://www.yes24.com/Product/Goods/2752350
                              ...                      
188959    https://www.yes24.com/Product/Goods/117650984
188960    https://www.yes24.com/Product/Goods/117585533
188961    https://www.yes24.com/Product/Goods/115033554
188962    https://www.yes24.com/Product/Goods/114848694
188963    https://www.yes24.com/Product/Goods/113416787
Name: link, Length: 188964, dtype: object

# 셀레니움 상세페이지 크롤링

In [28]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
from selenium.common.exceptions import NoSuchElementException

In [42]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절

In [87]:
driver.get("https://www.yes24.com/Product/Goods/114848694")

In [44]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight * 0.75);")  
driver.execute_script("window.scrollTo(0,document.body.scrollHeight * 0.83);")  

In [ ]:
detail_page = []

for links in data["link"]:

    # driver는 해상도에 맞춰서 빈 사이트를 띄워주는 로직 
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
        driver.get(links) # for 루프 안에 있는 link를 하나씩 넣으면서 페이지를 띄우게함 

        time.sleep(2)# 2초 쉬었다가 
        down_button = driver.find_element(By.CSS_SELECTOR,"#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(3) > a")
        down_button.click()
        # 댓글을 수집하기 위해서 배송/반품/교환 섹션으로 내려가게끔 페이지 다운 버튼 클릭 
        time.sleep(2)
        # 페이지를 댓글 창으로 끌어 올리기 위해서 body를 변수에 넣음 
        body = driver.find_element(By.CSS_SELECTOR,"body")
        # body변수를 page_up 누른것처럼 페이지를 살짝 올려주는 로직 
        body.send_keys(Keys.PAGE_UP)
        time.sleep(3)
        # 도착한 포커스 내에서 페이지를 모두 저장 시킴(html_page라는 변수에)
        html_page = driver.page_source
        # 그 수집한 html_page 변수를 detail_page라는 리스트에 저장 시킴 
        detail_page.append(html_page)
        # 그 이후에 페이지 변수를 증가시킴 
        page = 1 
        nth_child = 4
        time.sleep(3)
        while True:
            print(f"페이지 : {page}, nth-child : {nth_child}, 페이지 개수 {len(detail_page)}")
            time.sleep(2)
            next_btn = driver.find_element(By.CSS_SELECTOR,f"#infoset_oneCommentList > div:nth-child(4) > div.rvCmt_sortLft > div > a:nth-child({nth_child})")
            next_btn.click()
            time.sleep(2)
            nth_child += 1
            page +=1 
            html_page = driver.page_source
            detail_page.append(html_page)
            time.sleep(3)
            try: 
                empty_button = driver.find_element(By.CSS_SELECTOR,"#infoset_oneCommentList > div:nth-child(4) > div.rvCmt_sortLft > div > a.bgYUI.next.dim")
                break
            except NoSuchElementException:
                pass

            if page % 10 == 0:
                time.sleep(2)
                next_prev_button = driver.find_element(By.CSS_SELECTOR,"#infoset_oneCommentList > div:nth-child(4) > div.rvCmt_sortLft > div > a.bgYUI.next")
                next_prev_button.click()
                page += 1 
                nth_child = 4
                html_page = driver.page_source
                detail_page.append(html_page)
                time.sleep(3)
            
    except Exception :
        print(Exception)
        pass
    
    finally:
        driver.quit()
            
        # DB설계를 하고 해야함..
        # Book_id를 설정해야하는데 어케하는지모름

페이지 : 1, nth-child : 4, 페이지 개수 1
페이지 : 2, nth-child : 5, 페이지 개수 2
페이지 : 3, nth-child : 6, 페이지 개수 3
페이지 : 4, nth-child : 7, 페이지 개수 4
페이지 : 5, nth-child : 8, 페이지 개수 5
페이지 : 6, nth-child : 9, 페이지 개수 6
페이지 : 7, nth-child : 10, 페이지 개수 7
페이지 : 8, nth-child : 11, 페이지 개수 8
페이지 : 9, nth-child : 12, 페이지 개수 9
페이지 : 11, nth-child : 4, 페이지 개수 11
페이지 : 12, nth-child : 5, 페이지 개수 12
페이지 : 13, nth-child : 6, 페이지 개수 13
페이지 : 14, nth-child : 7, 페이지 개수 14
페이지 : 15, nth-child : 8, 페이지 개수 15
페이지 : 16, nth-child : 9, 페이지 개수 16
페이지 : 1, nth-child : 4, 페이지 개수 18
페이지 : 1, nth-child : 4, 페이지 개수 20
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 21
페이지 : 2, nth-child : 5, 페이지 개수 22
페이지 : 3, nth-child : 6, 페이지 개수 23
페이지 : 4, nth-child : 7, 페이지 개수 24
페이지 : 5, nth-child : 8, 페이지 개수 25
페이지 : 1, nth-child : 4, 페이지 개수 27
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 28
페이지 : 1, nth-child : 4, 페이지 개수 30
페이지 : 1, nth-child : 4, 페이지 개수 32
페이지 : 1, nth-child : 4, 페이지 개수 34
<class 'Exception'>
페이지 : 1, nth-child : 4

페이지 : 5, nth-child : 8, 페이지 개수 290
페이지 : 6, nth-child : 9, 페이지 개수 291
페이지 : 1, nth-child : 4, 페이지 개수 293
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 294
페이지 : 2, nth-child : 5, 페이지 개수 295
페이지 : 1, nth-child : 4, 페이지 개수 297
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 298
페이지 : 1, nth-child : 4, 페이지 개수 300
페이지 : 1, nth-child : 4, 페이지 개수 302
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 303
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 304
페이지 : 1, nth-child : 4, 페이지 개수 306
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 307
페이지 : 1, nth-child : 4, 페이지 개수 309
페이지 : 1, nth-child : 4, 페이지 개수 311
페이지 : 1, nth-child : 4, 페이지 개수 313
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 314
페이지 : 2, nth-child : 5, 페이지 개수 315
페이지 : 3, nth-child : 6, 페이지 개수 316
페이지 : 4, nth-child : 7, 페이지 개수 317
페이지 : 5, nth-child : 8, 페이지 개수 318
페이지 : 6, nth-child : 9, 페이지 개수 319
페이지 : 1, nth-child : 4, 페이지 개수 321
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 322
<class 'Exception'>


페이지 : 1, nth-child : 4, 페이지 개수 550
페이지 : 1, nth-child : 4, 페이지 개수 552
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 553
페이지 : 2, nth-child : 5, 페이지 개수 554
페이지 : 3, nth-child : 6, 페이지 개수 555
페이지 : 4, nth-child : 7, 페이지 개수 556
페이지 : 5, nth-child : 8, 페이지 개수 557
페이지 : 6, nth-child : 9, 페이지 개수 558
페이지 : 7, nth-child : 10, 페이지 개수 559
페이지 : 8, nth-child : 11, 페이지 개수 560
페이지 : 9, nth-child : 12, 페이지 개수 561
페이지 : 11, nth-child : 4, 페이지 개수 563
페이지 : 1, nth-child : 4, 페이지 개수 565
페이지 : 1, nth-child : 4, 페이지 개수 567
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 568
페이지 : 1, nth-child : 4, 페이지 개수 570
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 571
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 572
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 573
페이지 : 1, nth-child : 4, 페이지 개수 575
페이지 : 2, nth-child : 5, 페이지 개수 576
페이지 : 1, nth-child : 4, 페이지 개수 578
페이지 : 1, nth-child : 4, 페이지 개수 580
페이지 : 1, nth-child : 4, 페이지 개수 582
페이지 : 1, nth-child : 4, 페이지 개수 584
<class 'Exception'>
페

페이지 : 1, nth-child : 4, 페이지 개수 813
페이지 : 1, nth-child : 4, 페이지 개수 815
페이지 : 2, nth-child : 5, 페이지 개수 816
페이지 : 3, nth-child : 6, 페이지 개수 817
페이지 : 4, nth-child : 7, 페이지 개수 818
페이지 : 5, nth-child : 8, 페이지 개수 819
페이지 : 6, nth-child : 9, 페이지 개수 820
페이지 : 7, nth-child : 10, 페이지 개수 821
페이지 : 8, nth-child : 11, 페이지 개수 822
페이지 : 9, nth-child : 12, 페이지 개수 823
페이지 : 11, nth-child : 4, 페이지 개수 825
페이지 : 12, nth-child : 5, 페이지 개수 826
페이지 : 13, nth-child : 6, 페이지 개수 827
페이지 : 14, nth-child : 7, 페이지 개수 828
페이지 : 15, nth-child : 8, 페이지 개수 829
페이지 : 1, nth-child : 4, 페이지 개수 831
페이지 : 1, nth-child : 4, 페이지 개수 833
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 834
페이지 : 1, nth-child : 4, 페이지 개수 836
페이지 : 1, nth-child : 4, 페이지 개수 838
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 839
페이지 : 1, nth-child : 4, 페이지 개수 841
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 842
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 843
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 844
페이지 : 1, nth-chil

페이지 : 19, nth-child : 12, 페이지 개수 1069
페이지 : 21, nth-child : 4, 페이지 개수 1071
페이지 : 22, nth-child : 5, 페이지 개수 1072
페이지 : 23, nth-child : 6, 페이지 개수 1073
페이지 : 24, nth-child : 7, 페이지 개수 1074
페이지 : 25, nth-child : 8, 페이지 개수 1075
페이지 : 26, nth-child : 9, 페이지 개수 1076
페이지 : 27, nth-child : 10, 페이지 개수 1077
페이지 : 28, nth-child : 11, 페이지 개수 1078
페이지 : 29, nth-child : 12, 페이지 개수 1079
페이지 : 31, nth-child : 4, 페이지 개수 1081
페이지 : 32, nth-child : 5, 페이지 개수 1082
페이지 : 33, nth-child : 6, 페이지 개수 1083
페이지 : 34, nth-child : 7, 페이지 개수 1084
페이지 : 35, nth-child : 8, 페이지 개수 1085
페이지 : 36, nth-child : 9, 페이지 개수 1086
페이지 : 37, nth-child : 10, 페이지 개수 1087
페이지 : 38, nth-child : 11, 페이지 개수 1088
페이지 : 1, nth-child : 4, 페이지 개수 1090
페이지 : 1, nth-child : 4, 페이지 개수 1092
페이지 : 1, nth-child : 4, 페이지 개수 1094
페이지 : 1, nth-child : 4, 페이지 개수 1096
페이지 : 1, nth-child : 4, 페이지 개수 1098
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 1099
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 1100
<class 'Exception'>
페이지 : 1, nth-chi

...

<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5682
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5683
페이지 : 1, nth-child : 4, 페이지 개수 5685
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5686
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5687
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5688
페이지 : 1, nth-child : 4, 페이지 개수 5690
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5691
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5692
페이지 : 1, nth-child : 4, 페이지 개수 5694
페이지 : 1, nth-child : 4, 페이지 개수 5696
페이지 : 1, nth-child : 4, 페이지 개수 5698
페이지 : 1, nth-child : 4, 페이지 개수 5700
페이지 : 1, nth-child : 4, 페이지 개수 5702
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5703
페이지 : 1, nth-child : 4, 페이지 개수 5705
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5706
페이지 : 1, nth-child : 4, 페이지 개수 5708
페이지 : 2, nth-child : 5, 페이지 개수 5709
페이지 : 1, nth-child : 4, 페이지 개수 5711
페이지 : 1, nth-child : 4, 페이지 개수 5713
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5714
페이지 : 1,

페이지 : 1, nth-child : 4, 페이지 개수 5946
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5947
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5948
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5949
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5950
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5951
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5952
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5953
페이지 : 1, nth-child : 4, 페이지 개수 5955
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 5956
페이지 : 1, nth-child : 4, 페이지 개수 5958
페이지 : 1, nth-child : 4, 페이지 개수 5960
페이지 : 2, nth-child : 5, 페이지 개수 5961
페이지 : 3, nth-child : 6, 페이지 개수 5962
페이지 : 4, nth-child : 7, 페이지 개수 5963
페이지 : 5, nth-child : 8, 페이지 개수 5964
페이지 : 6, nth-child : 9, 페이지 개수 5965
페이지 : 7, nth-child : 10, 페이지 개수 5966
페이지 : 8, nth-child : 11, 페이지 개수 5967
페이지 : 9, nth-child : 12, 페이지 개수 5968
페이지 : 11, nth-child : 4, 페이지 개수 5970
페이지 : 12, nth-child : 5, 페이지 개수 5971
페이지 : 13, nth-child : 6, 페이지 개수 5972
페이지 : 

페이지 : 2, nth-child : 5, 페이지 개수 6209
페이지 : 3, nth-child : 6, 페이지 개수 6210
페이지 : 4, nth-child : 7, 페이지 개수 6211
페이지 : 5, nth-child : 8, 페이지 개수 6212
페이지 : 6, nth-child : 9, 페이지 개수 6213
페이지 : 7, nth-child : 10, 페이지 개수 6214
페이지 : 8, nth-child : 11, 페이지 개수 6215
페이지 : 1, nth-child : 4, 페이지 개수 6217
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6218
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6219
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6220
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6221
페이지 : 1, nth-child : 4, 페이지 개수 6223
페이지 : 1, nth-child : 4, 페이지 개수 6225
페이지 : 1, nth-child : 4, 페이지 개수 6227
페이지 : 1, nth-child : 4, 페이지 개수 6229
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6230
페이지 : 1, nth-child : 4, 페이지 개수 6232
페이지 : 2, nth-child : 5, 페이지 개수 6233
페이지 : 3, nth-child : 6, 페이지 개수 6234
페이지 : 4, nth-child : 7, 페이지 개수 6235
페이지 : 5, nth-child : 8, 페이지 개수 6236
페이지 : 6, nth-child : 9, 페이지 개수 6237
페이지 : 7, nth-child : 10, 페이지 개수 6238
페이지 : 8, nth-child : 11, 페이지 개수 6

페이지 : 1, nth-child : 4, 페이지 개수 6480
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6481
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6482
페이지 : 1, nth-child : 4, 페이지 개수 6484
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6485
페이지 : 1, nth-child : 4, 페이지 개수 6487
페이지 : 1, nth-child : 4, 페이지 개수 6489
페이지 : 1, nth-child : 4, 페이지 개수 6491
페이지 : 1, nth-child : 4, 페이지 개수 6493
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6494
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6495
페이지 : 1, nth-child : 4, 페이지 개수 6497
페이지 : 1, nth-child : 4, 페이지 개수 6499
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6500
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6501
페이지 : 1, nth-child : 4, 페이지 개수 6503
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6504
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6505
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6506
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6507
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6508
페이지 

페이지 : 25, nth-child : 8, 페이지 개수 6761
페이지 : 1, nth-child : 4, 페이지 개수 6763
페이지 : 1, nth-child : 4, 페이지 개수 6765
페이지 : 2, nth-child : 5, 페이지 개수 6766
페이지 : 3, nth-child : 6, 페이지 개수 6767
페이지 : 4, nth-child : 7, 페이지 개수 6768
페이지 : 5, nth-child : 8, 페이지 개수 6769
페이지 : 6, nth-child : 9, 페이지 개수 6770
페이지 : 7, nth-child : 10, 페이지 개수 6771
페이지 : 8, nth-child : 11, 페이지 개수 6772
페이지 : 9, nth-child : 12, 페이지 개수 6773
페이지 : 11, nth-child : 4, 페이지 개수 6775
페이지 : 12, nth-child : 5, 페이지 개수 6776
페이지 : 13, nth-child : 6, 페이지 개수 6777
페이지 : 14, nth-child : 7, 페이지 개수 6778
페이지 : 15, nth-child : 8, 페이지 개수 6779
페이지 : 1, nth-child : 4, 페이지 개수 6781
페이지 : 1, nth-child : 4, 페이지 개수 6783
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6784
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6785
페이지 : 1, nth-child : 4, 페이지 개수 6787
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6788
페이지 : 1, nth-child : 4, 페이지 개수 6790
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지 개수 6791
<class 'Exception'>
페이지 : 1, nth-child : 4, 페이지

In [24]:
len(detail_page)

20

In [60]:
soup.select("#yesBestList > li:nth-child(1) > div > div.item_info > div.info_row.info_name > a.gd_name")

[<a class="gd_name" href="/Product/Goods/2599166" onclick="wiseLogV2('BS', '004_007_001', ''); ">The Secret 시크릿</a>]

In [268]:
html_page = driver.page_source

In [75]:
soup2 = bs(html_page,"lxml")

In [278]:
for i in soup2.select(".cmtInfoBox"):
    print(i.select_one(".cmt_cont>.txt").text)

좋아요
너무너무 명작입니다
추천
추천합니다
신형철의 새책
감사합니다 열심히 읽게습니다.
인생의 역사를 알게해준다
단연!
신형철이 써내려간 연과 행, 이번에도 기꺼이 연행된다.
어렵게만 느껴졌던 시와 약간은 친해진 느낌이에요. 신형철님 책은 항상 근사합니다.
긴 기다림의 끝에 오는 즐거움을 알게 해주는 작가를 만난다는 것은 큰 행운이다.
계속 잘 읽어왔기에 기대하며 구매합니다. 쉽게 읽히지만 깊은 언어를 추앙합니다.
너무 좋은 책이다 이것을 읽고 있는 나 자신이 좋아질 정도로..
어디를 펴도 좋습니다
다 읽어가는게 아쉬워 아껴 읽습니다
좋아하는 분들께 선물하고 있습니다
